**[Do not edit the contents of this cell]**

# MSc in Bioinformatics and Theoretical Systems Biology - Maths and Stats Assignment 2020/21

This assignment is to be completed in Python, R or Julia and returned as a clean Jupyter notebook cleared of its output. This is important otherwise Turnitin will reject the submission.

There are 4 types of cells used in this notebook:
1. Cells containing tasks and instructions to be completed. Do not edit these. These are clearly labelled.
2. Cells in which you are meant to provide an answer in Markdown format.
3. Cells containing code that defines e.g. which packages to load, but can also contain routines and snippets of codes that you should use.
4. Cells that contain the Python/R/Julia code that you write to solve the problems set.

Each of these cells will contain explit comments at the top telling you whether to edit or not edit a cell. In Code cells comments are specified by the "#" character. In the Markdown Answer Cells, replace the xxx by your answer, whenever these are present. You will have to execute all code and Markdown cells in order to (i) make use of the provided code, and (ii) format the markdown appropriately.

There are four problems to be tackled:
1. Data exploration [40%]
2. Hypothesis testing [20%]
3. Regression [20%]
4. Classification [20%]

For each questions there several parts of different difficulty. Where appropriate, further reading will be given at the start of each question.

You will have to specify which language (and version) you used and all packages needed in order to run all Code cells. Please add this information in the next two cells.

**[Provide your answer here]**
- The kernel for this Jupyter notebook is R, version 3.6.1, with the following packages: ggplot2, fitdistrplus, dplyr, e1071, caret

In [ ]:
library('ggplot2')
library('fitdistrplus')
library('dplyr')
library('e1071')
library('caret')

**[Do not edit the contents of this cell]**

## Problem 1: data exploration

We consider a subset of data coming from a putative association study where researchers collected various metrics and phenotypes to find associations with a putative generic cardiovascular disease.
All recruited subjects are adults.

For each subject several __predictor variables__ are recorded: sex (1 for female, 0 for male), height (in cm), mass (in kg), whether he/she is a smoker (1) or not (0), whether he/she is a drinker (1) or not (0), and levels of 5 different metabolites (labelled A-E).
**Each subject has a unique ID number**. 

For each subject a disease score, which is the __response__ variable, measuring the severity of the disease phenotype in arbitrary units, is provided.
The data is provided in the file `association_data.csv`.

### Part 1

Load the dataset `association_data.csv`.
How many unique records of subjects do we have? How many unique predictor variables?

In [ ]:
#setwd("~/Imperial/Maths/Stats/Assignment")
asso = read.csv('association_data.csv')

summary(asso)

for (attribute in names(asso)){
  ua = length(unique(asso[,attribute]))
  cat(ua, "unique", attribute, "instances", "\n")
}

#note: height is given in cm!

**[Provide your answer here]**
- The number of unique records of subjects in the dataset is: 1000
- The number of variables in the dataset is: 12 (but 11 predictor variables)

**[Do not edit the contents of this cell]**

### Part 2

Produce a plot to illustrate the distribution of variables `smoker`, `mass`, `metabolite_A`. Choose the most appropriate visualisation depending on the type of each variable.

In [ ]:
smoke_frame = data.frame(Smoke = c(sum(asso$smoker),(length(asso$smoker) - sum(asso$smoker))),
                         Labels = c("Smoker", "Non-smoker"))
smoke_frame$perc = smoke_frame$Smoke / length(asso$smoke) * 100


ggplot(smoke_frame, aes(x="", y=Smoke, fill = Labels)) +
  geom_bar(stat="identity", width=1, color="white") +
  coord_polar("y", start=0) + 
  theme_void() +
  geom_text(aes(y = c(100, 700), label = perc)) +
  labs(title = "Smoker Distribution", y = "Smoker")


In [ ]:
#The following two functions are taken from a data modelling assignment
#I took in my undergrad. They allow me to easily plot histograms as well as calculate distribution skew.

calc_skew = function(my_dat, my_mean, my_median, my_sdev, type){ #returns skewness of dist
  skew = (3*(my_mean - my_median))/my_sdev
}

create_hist = function(dframe,atrib,Titl,width,se){
  my_dat = dframe[,atrib]
  my_dat = my_dat[!is.na(my_dat)] # remove missing vals
  my_min = min(my_dat)
  my_min = signif(my_min,3)
  my_max = max(my_dat)
  my_max = signif(my_max,3)
  
  my_mean = mean(my_dat) #calculate summary stats for skew and additional plot elements
  my_median = median(my_dat)
  my_sdev = sd(my_dat)
  
  my_skew = calc_skew(my_dat, my_mean, my_median, my_sdev, 2)
  print(my_skew)
  
  my_dat = as.data.frame(my_dat)
  
  ggplot(my_dat, aes(x=my_dat, color = "Lines")) + 
    scale_x_continuous(breaks = seq(floor(my_min), ceiling(my_max),by=se),atrib)+
    geom_histogram(binwidth = width, col = "Blue") +
    geom_vline(aes(xintercept=my_mean, col="Mean")) +
    geom_vline(aes(xintercept=my_median, col="Median"))+
    labs(title = Titl, y = "Frequency")
  
}

create_hist(asso,'mass', "Mass Distribution", 1, 2)

In [ ]:
create_hist(asso,'metabolite_A', "Metabolite A Distribution", 0.01,0.1)

**[Do not edit the contents of this cell]**

### Part 3

Write a function that returns the Body Mass Index (BMI). Learn from a literature search how to calculate BMI.
Calculate BMI for each subject and add it as new __predictor variable__ in the data set.

In [ ]:
asso$bmi = asso$mass / (asso$height/100)^2

**[Do not edit the contents of this cell]**

### Part 4

Calculate the correlation matrix between __quantitative__ (numerical) predictors. Use this information to impute any missing values, if possible.

In [ ]:
### Produce the correlation matrix ###
cor(asso[,2:length(asso)], use= 'pairwise.complete.obs')

### Gen Summary Statistics to see number of NA's ##

summary(asso) # see clearly that there's 2 NA's in metabolite_A

In [ ]:
reg = lm(metabolite_A~metabolite_B, data = asso)["coefficients"] # Regression model gives us our y = mx + c line between met a and b

reg_grad= reg$coefficients[1]
reg_int = reg$coefficients[2]

new_A = asso[is.na(asso["metabolite_A"]),"metabolite_B"] * reg_grad + reg_int

for (A in new_A){
  asso[is.na(asso["metabolite_A"]),"metabolite_A"] <- A
}

summary(asso) # shows missing values have been removed

**[Do not edit the contents of this cell]**

### Part 5

Assuming that a disease status is recorded when the disease score is greater than 1, add a new __response variable__ in the dataset defining the diseased `status` of each subject, either non diseased (0) or diseased (1).

In [ ]:
asso$status = ifelse((asso[,"disease_score"] > 1),1,0)

**[Do not edit the contents of this cell]**

## Problem 2: hypothesis testing

Starting from the same dataset in Problem 1, provide answers for the following questions.

### Part 1

Given this sample space of subjects, what is the probability that a given subject is diagnosed as diseased? What is the probability that a subject is diagnosed as diseased given that he/she is not a smoker?

In [ ]:
#Find the probability of diseased given sample set

prob_diseased = sum(asso$status)/nrow(asso)


#probability of diseased given the fact they smoke
smokers = asso[asso$smoker == 1,]

prob_dis_smoker = sum(smokers$status)/nrow(smokers)

prob_diseased
prob_dis_smoker


**[Provide your answer here]**
- The probability that a subject is diseased is: 0.383
- The probability that a subject is diseased given that she/he is not a smoker is: 0.502 (to 3sf)

**[Do not edit the contents of this cell]**

### Part 2

Assuming that they distributed as a Normal distribution $N(\mu, \sigma^2)$, provide an estimate of $\mu$ and $\sigma^2$ for the distributions of height and mass separately for males and females.

In [ ]:
get_param_estimates = function(dat, dist){
  mea = mean(dat)
  sdev = sd(dat)
  fd = fitdist(dat, dist, method = "mle")
  fit_mea = fd$estimate[1]
  fit_sd = fd$estimate[2]
  
  param_estimates = data.frame("Frequentist estimate" = c(mea,sdev),
                               "Maximum Likelihood Estimate" = c(fit_mea,fit_sd))
  rownames(param_estimates) = c("Mean", "Standard Deviation") 
  
  return(param_estimates)
}

males = asso[asso$sex == 0,]
females = asso[asso$sex == 1,]

#Estimate params:
print("Male Height Parameter Estimates")
get_param_estimates(males$height, "norm")
print("Female Height Parameter Estimates")
get_param_estimates(females$height, "norm")

print("Male Mass Parameter Estimates")
get_param_estimates(males$mass, "norm")
print("Female Mass Parameter Estimates")
get_param_estimates(females$mass, "norm")

**[Do not edit the contents of this cell]**

### Part 3

Test whether height is different between males and females. Perform the same test on the mass variable. Define (in words) which ones are your null and alternative hypotheses and significance threshold. Finally, discuss (in words) any conclusion you can make out the results of your statistical tests.

In [ ]:
height_t = t.test(males$height, females$height)
mass_t = t.test(males$mass, females$mass)

height_t
mass_t

**[Provide your answer here]**

- T Test Height: <br>
Null Hypothesis --> no difference in mean of female height and mean of male height <br>
Alternative Hypothesis --> difference in mean of female height and mean of male height not equal to zero <br>
Significance Threshold --> 0.05<br>
Conclusion --> since p is > 0.05 we must accept the null hypothesis (i.e. there is no difference in female and male height means)<br>
<br>
- T Test Mass: <br>
Null Hypothesis --> no difference in mean of female mass and mean of male mass <br>
Alternative Hypothesis --> difference in mean of female mass and mean of male mass not equal to zero <br>
Significance Threshold --> 0.05 <br>
Conclusion --> since p < 0.05, we can reject the null hypothesis (i.e. there is a significant difference between female and male weight means) <br>
    


**[Do not edit the contents of this cell]**

### Part 4

Repeat the statistical test in Part 3 for all numerical predictor variables in the dataset. How many tests are significant with $\alpha=0.05$?
Calculate corrected p-values for multiple tests using a Bonferroni correction. 
Learn what the Bonferroni correction implies from a literature search. 
How many tests are significant after correcting for multiple tests? 

In [ ]:
alpha = 0.05
test_frame = c()

for(atrib in names(asso)[3:length(names(asso))]){ #skip sex and id
  itt_test = t.test(males[,atrib], females[,atrib])
  itt_p = itt_test$p.value
  itt_row = data.frame(p = itt_p)
  test_frame = rbind(test_frame, itt_row)
}

test_frame$significance = ifelse((test_frame[,"p"] <= alpha),T,F)
row.names(test_frame) = names(asso)[3:length(names(asso))]
print("Significant Results: ")
test_frame[test_frame$significance == T,] #shows mass and bmi have significant differences in mean

### Bonferoni Correction ###

test_frame$bonferoni = ifelse((test_frame[,"p"] <= alpha/ncol(test_frame)),T,F)
print("Significant Results (bonferoni correction): ")
test_frame[test_frame$bonferoni == T,]

**[Do not edit the contents of this cell]**

## Problem 3: regression

Assume you have been provided with a new __predictor variable__ indicating a generic glucose index level. We know that such glucose index is related to the variable `metabolite_D` given in the previous dataset.

### Part 1

Load the new dataset called `glucose_data.csv` which gives the measure of a generic glucose level (`glucose_index`) for each tested subject. Perform a regression where `metabolite_D` is the predictor and `glucose_index` is the response variable. Be aware to merge the two data sets by matching subject IDs.

In [ ]:
setwd("~/Imperial/Maths/Stats/Assignment")
asso = read.csv("asso.csv")
gluc = read.csv("glucose_data.csv")

asso = left_join(asso, gluc, by="subject")


lin_mod = lm(glucose_index~metabolite_D,data = asso)
#summary(lin_mod)

int = lin_mod$coefficients[1]
grad = lin_mod$coefficients[2]
print("Formula: ")
cat("glucose = metabolite_D * ",grad, " + ",int)

**[Do not edit the contents of this cell]**

## Problem 4: classification

### Part 1

Implement an algorithm to predict the disease status of a subject given all response variables provided in the dataset. You are free to choose the appropriate statistical tool you prefer. Assess the performance of your classifier.

In [ ]:
set.seed(42)


partition = function(dset){ #return list of train and test sets
  train_ind = createDataPartition(y= dset$status, p = 0.75, list = FALSE)
  train = dset[train_ind,]
  test = dset[-train_ind,]
  ret = list(train, test)
  return(ret)
} 



## Base dataset - no reduction
base = asso #Store a base dataset with no dim reduction
base[,c("subject","disease_score")] = NULL #remove the subject and disease score vars



## dr set - delete by weak correlation to response var
stat_cors = cor(asso[,2:length(asso)], use ='pairwise.complete.obs')[,"status"] 
threshold = 0.2
to_del = names(stat_cors[abs(stat_cors) < threshold]) #get list of vars to remove from df 

dr = asso
for(name in to_del){
  dr[,name] = NULL
}
dr[,c("disease_score","subject")] = NULL

## PC set - combine into PC's covering > ... variance of dataset
pc = asso
status = asso$status
pc[,c("subject","status","disease_score")] = NULL
pc = as.data.frame(scale(pc))
pc = prcomp(pc,scale=T)
#pc
pc = as.data.frame(pc$x[,1:7]) #contains > 95% of variance of set, uncomment to see this
pc = cbind(pc, status)
###

gen_svm = function(dat){
  
  #partition and format
  p_dat = partition(dat)
  
  train_dat = as.data.frame(p_dat[1])
  train_dat$status = as.factor(train_dat$status)
  test_dat = as.data.frame(p_dat[2])
  test_dat$status = as.factor(test_dat$status)
  
  svm_dat = svm(status ~ ., data = train_dat, kernel = "radial", cost =5, scale = F)
  svm_pred = predict(svm_dat, test_dat)
  
  ret = data.frame(status = test_dat$status, pred = svm_pred)
  cmat = confusionMatrix(data = ret$pred, reference = ret$status)
  print(cmat)
  return(svm_dat)
  
}
print("Base Dataset SVM Results: ")
base_SVM = gen_svm(base)
print("Dimensionality Reduced Dataset SVM Results: ")
dr_SVM = gen_svm(dr)
print("Principal Component Reduced Dataset SVM Results: ")
pc_SVM = gen_svm(pc)


**[Do not edit the contents of this cell]**

### Part 2

Given the classifier you devised in Part 1, predict the disease status of the following subject:
- subject: ID1986
- sex: 1
- height: 180.2
- mass: 70.1
- smoker: 1
- drinker: 0
- metabolite_A: 0.5
- metabolite_B: 1.2
- metabolite_C: 0.5
- metabolite_D: 8.5
- metabolite_E: 10.2

In [ ]:
new_subject = data.frame(subject = "ID1986",
                        sex = 1,
                        height = 180.2,
                        mass=70.1,
                        smoker=1,
                        drinker= 0,
                        metabolite_A = 0.5,
                        metabolite_B= 1.2,
                        metabolite_C= 0.5,
                        metabolite_D= 8.5,
                        metabolite_E= 10.2)

new_subject$bmi = new_subject$mass / (new_subject$height/100)^2

predict(dr_SVM, new_subject) #subject is diseased!


**[Provide your answer here]**

<h2>Preprocessing </h2><br>
For this exercise several datasets were generated in an attempt to attain the best performing classifier. The first of these was the base dataset - a set that contains all variables appart from the subject ID and the disease score (since status is directly calculated from the disease score, It seemed redundant to retain it in our dataset - the model would be classifying based on high/low disease score rather than the underlying relationship between the predictor variables). <br> 
<br>
The second dataset was a dimensionality reduced set. This dataset removed variables that weakly correlated with the class variable (the idea being that these variables were uninformative so removing them helped to aleviate the curse of dimensionality allowing for more accurate classification). <br>
<br>
The Third dataset contained the first 7 principal components of the dataset. This accounted for ~95% of the variation in the data, meaning I could aleviate the curse of dimensionality while retaining the majority of the variance.This was used to help to confirm the validity of the dimensionality reduced set<br>
<br>
<h2> Classifier</h2><br>
For this exercise, all classification was performed using Support Vector Machines <br>
<br>

<h2> Evaluating Classification: </h2><br>
    
The classifier produced by the Base dataset performed well with an accuracy of \~81%. When analysing the confusion matrix it is apparent that there is a bias towards false negative predictions (since there are 30 false negative predictions but only 18 false positive predictions). <br>
<br>
The classifier produced by the Dimensionality reduced dataset on the other hand vastly outperformed the one created by the Base dataset with an accuracy of \~93%. Unlike the Base set classifier, it does not seem to show a similar bias with and equal number of false positives and false negatives. <br>
<br>
The classifier produced by the PCA dataset shows a very similar accuracy to the Dimensionality Reduced classifier (~92%) which implies that, despite only retaining 3 variables, it is able to produce models that encompas ~95% of the total variation of the dataset thus validating the decision to excise the other variables from the set. <br>  
<br>
<h2>Categorising the new subject:</h2> <br>
<br>
Using the best performing classifier from pt1 (the dimensionality reduced Support Vector Machine), I was able to classify the new instance as diseased